In [5]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
       
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    ### END CODE HERE ###
    
    return loss

In [6]:
import pickle

FRmodel = pickle.load( open( "FRmodel.p", "rb" ) )

ValueError: Unknown loss function:triplet_loss

In [ ]:
def img_to_encoding(image_path, model):

    input_dim=350 # Ridge image input dimensions are (350,350). Only taking the height and width. Actually it is (350,350,3)including RGB.
    facenet_dim=96 # FaceNet model uses an input of (96,96). To take advantage of preset weights, converts Ridge images to multiple FaceNet size images

    no_of_cut=math.ceil(input_dim/facenet_dim)

    img = cv2.imread(image_path, 1)

    img=np.pad(img,((1,1),(1,1),(0,0)),'constant') # Padding so that the image dimension is divisible by the no_of cut

    mod_input_dim = img.shape[0]

    cut_dim=int(mod_input_dim/no_of_cut) # This is the dimension of the image that is cut. To make it equal to 96, padding with zeros are done.
    crop_pad=int((facenet_dim-cut_dim)/2 )# Padding equal number of zeros on both sides(up-down,left-right) to make the cut dim (88) equal to facenet dim(96)
    cropped_images=[]

    for y in range(no_of_cut):
        for x in range(no_of_cut):
            x_start=x*cut_dim
            x_end=(x+1)*cut_dim
            y_start=y*cut_dim
            y_end=(y+1)*cut_dim
            img_part=img[x_start:x_end,y_start:y_end,:]
            img_part=np.pad(img_part,((crop_pad,crop_pad),(crop_pad,crop_pad),(0,0)),'constant')
            cropped_images.append(img_part)

    embeddings=None
    for imag in cropped_images:
        img = imag[...,::-1]
        img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
        x_train = np.array([img])
        embedding = model.predict_on_batch(x_train)
        if embeddings is None:
            embeddings=embedding
        else:
            embeddings=np.concatenate((embeddings,embedding),axis=1)
        flat_input=embedding.flatten()
    
    return embeddings

In [ ]:
def show_images(image_path_original,image_path_check):
    
    imageA=mpimg.imread(image_path_original)
    imageB=mpimg.imread(image_path_check)
    fig = plt.figure("Comparison")
 
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the images
    plt.show()

In [ ]:
def verify(image_path_original, image_path_check, model): 
    
    encoding_original = img_to_encoding(image_path_original,model)
    encoding_check = img_to_encoding(image_path_check,model)
    
    dist = np.linalg.norm(encoding_original-encoding_check)
 #   dist = dist/256
    # setup the figure
    
    show_images(image_path_original,image_path_check)
    print ("Distance is " , dist)
    if dist < 1:
        print("It's similar")          
    else:
        print("It's not similar")
    

In [ ]:
def verify_ml_approach(image_path_original,image_path_check):
    verify(image_path_original, image_path_check, FRmodel)

In [ ]:
def load_train_data():
    dataset = np.genfromtxt("data_from_prg.txt", dtype="U" ,delimiter="," )
    X = dataset[:,0:2]
    Y = dataset[:,2]
    Y= [int(i) for i in Y]
    m=len(Y)
    return X,Y,m

In [ ]:
def prepare_train_data(X,Y,m,classes=11):
    dat=np.zeros((1,4096))
    data=np.zeros((m,4096))
    labels=np.zeros((m,classes))
    row=0
    for [ori,chk] in X:
        ori_encoding=img_to_encoding(ori, FRmodel)
        chk_encoding=img_to_encoding(chk, FRmodel)
        dat=np.concatenate((ori_encoding,chk_encoding),axis=1)
        data[row,:]=dat
        labels[row,Y[row]]=1
        row=row+1  
    return data,labels

In [ ]:
def getSimpleModel():
    inputs=Input(shape=(4096,))
    # a layer instance is callable on a tensor, and returns a tensor
    x = Dense(1024, activation='relu')(inputs)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(11, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
def train_2nd_model():
    X,Y,m=load_train_data()
    data,labels=prepare_train_data(X,Y,m)

    Smodel=getSimpleModel()
    Smodel.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    Smodel.fit(data,labels,epochs=50 )  # starts training
    score = Smodel.evaluate(data,labels)
    return Smodel
    

In [ ]:
import pickle
Smodel=train_2nd_model()
pickle.dump(Smodel,open("SModel.p","wb"))
